In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

In [5]:
train['lgb_pred'] = np.load("../input/tps-aug-2021-lgb/lgb_oof.npy")
test['lgb_pred'] = np.load("../input/tps-aug-2021-lgb/lgb_pred.npy")

train['xgb_pred'] = np.load("../input/tps-aug-2021-xgb/xgb_oof.npy")
test['xgb_pred'] = np.load("../input/tps-aug-2021-xgb/xgb_pred.npy")

# Preprocessing

In [6]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [7]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [8]:
pseudo = pd.read_csv("../input/blending-tool-tps-aug-2021/blend.csv/0.part")[target]
test_pseudo = pd.concat([test, pseudo], axis=1)
all_pseudo = pd.concat([train, test_pseudo]).reset_index(drop=True)

In [9]:
useful_features = scale_features + ['lgb_pred', 'xgb_pred']

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo[useful_features], y=all_pseudo[target]):
  """
  """
  param_space = {
                    'objective': 'regression',
              'metric':'rmse',
               #'device':'gpu',  # Use GPU acceleration
               # 'gpu_platform_id': 0,
                #    'gpu_device_id': 0,
               'n_estimators':N_ESTIMATORS,
              'learning_rate':trial.suggest_uniform('learning_rate', 1e-3, 1e-1),
               'subsample': trial.suggest_uniform('subsample', 0.2, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.2, 1.0),
                'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
             #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 0.01, 50),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 0.01, 50),
            'min_child_weight':trial.suggest_uniform('min_child_weight', 100, 500),
                 'min_child_samples':trial.suggest_int('min_child_samples', 5, 250),
          'importance_type': 'gain'
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo[useful_features].iloc[trn_idx], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo[useful_features].iloc[oof_idx], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]

  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-26 02:46:22,215] A new study created in memory with name: no-name-fa0942ee-f890-472a-8b11-a4c64c4dc882


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[162]	valid_0's rmse: 7.74621
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[153]	valid_0's rmse: 7.74584


[I 2021-08-26 02:47:09,764] Trial 0 finished with value: 7.745670686888021 and parameters: {'learning_rate': 0.020487596550574833, 'subsample': 0.3762696077694573, 'colsample_bytree': 0.7811208091898296, 'subsample_freq': 5, 'reg_lambda': 29.462438771823344, 'reg_alpha': 41.37629922945654, 'min_child_weight': 486.8218095543472, 'min_child_samples': 167}. Best is trial 0 with value: 7.745670686888021.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[167]	valid_0's rmse: 7.74633
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[166]	valid_0's rmse: 7.74608


[I 2021-08-26 02:47:50,987] Trial 1 finished with value: 7.746072640176114 and parameters: {'learning_rate': 0.022503877232941553, 'subsample': 0.8938954390859548, 'colsample_bytree': 0.4934278485156569, 'subsample_freq': 1, 'reg_lambda': 29.13100669067273, 'reg_alpha': 1.656022586240838, 'min_child_weight': 171.472850264289, 'min_child_samples': 196}. Best is trial 0 with value: 7.745670686888021.


...

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[194]	valid_0's rmse: 7.74499
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[212]	valid_0's rmse: 7.74495


[I 2021-08-26 03:29:22,789] Trial 28 finished with value: 7.744875059307531 and parameters: {'learning_rate': 0.01387252397740572, 'subsample': 0.675661622929066, 'colsample_bytree': 0.9100887870664723, 'subsample_freq': 1, 'reg_lambda': 13.59596008963059, 'reg_alpha': 7.781177873195088, 'min_child_weight': 388.85854378357516, 'min_child_samples': 102}. Best is trial 21 with value: 7.744188593192304.


Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[165]	valid_0's rmse: 7.74578
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[180]	valid_0's rmse: 7.74584


[I 2021-08-26 03:30:18,603] Trial 29 finished with value: 7.74560852036608 and parameters: {'learning_rate': 0.017316613759359244, 'subsample': 0.5698172729896118, 'colsample_bytree': 0.8158179323029024, 'subsample_freq': 5, 'reg_lambda': 20.049687838022543, 'reg_alpha': 21.63540168936585, 'min_child_weight': 493.5244730280424, 'min_child_samples': 55}. Best is trial 21 with value: 7.744188593192304.


Number of finished trials: 30
Best trial: {'learning_rate': 0.010366474352623736, 'subsample': 0.6916816311493477, 'colsample_bytree': 0.9744058120317534, 'subsample_freq': 3, 'reg_lambda': 0.7706005781123387, 'reg_alpha': 5.8983343945165005, 'min_child_weight': 348.82810828234136, 'min_child_samples': 7}


In [13]:
study.best_params

{'learning_rate': 0.010366474352623736,
 'subsample': 0.6916816311493477,
 'colsample_bytree': 0.9744058120317534,
 'subsample_freq': 3,
 'reg_lambda': 0.7706005781123387,
 'reg_alpha': 5.8983343945165005,
 'min_child_weight': 348.82810828234136,
 'min_child_samples': 7}

# Log
